In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql("USE ROLE TRAINING_ROLE").collect()     # or TRAINING_ROLE
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()
records        = session.table("PERMIT_RECORDS").to_pandas()
census_tracts  = session.table("CENSUS_TRACTS").to_pandas()
contractors    = session.table("MASTER_LICENSE").to_pandas()
personnel      = session.table("PERSONNEL_DATA").to_pandas()
worker_comp    = session.table("WORKER_COMP").to_pandas()

In [ ]:
records = session.table('PERMIT_RECORDS').to_pandas()

In [ ]:
records.head(5)

In [ ]:
records.columns

In [ ]:
records["FLOAT_VALUATION"] = pd.to_numeric(
    records["VALUATION"].str.replace("$","").str.replace(",",""),errors='coerce')

In [ ]:
records["ISSUE_DATE"] = pd.to_datetime(records["ISSUE_DATE"])

In [ ]:
# seeing each permit type from the data
records["PERMIT_TYPE"].value_counts()

In [ ]:
# getting specific rows of data 
records[records["PERMIT_TYPE"] == "HVAC"]

In [ ]:
census_tracts = session.table('CENSUS_TRACTS').to_pandas()

In [ ]:
census_tracts.head(5)

In [ ]:
pd.to_numeric(records["CENSUS_TRACT"])*100

In [ ]:
records["CT"] = (pd.to_numeric(records["CENSUS_TRACT"])*100)+6037000000

In [ ]:
census_tracts = census_tracts[census_tracts["CENSUS_TRACT"].isin(records["CT"])]

In [ ]:
census_tracts.merge(records, left_on="CENSUS_TRACT", right_on="CT")

In [ ]:
#using the descriptions for getting data
records[records["AI_DESCRIPTION"].fillna("").str.contains("kitchen")]

In [ ]:
records[(records["STATUS"] == "Permit Finaled")&(records["AI_DESCRIPTION"].str.contains(
    "kitchen"))]

In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
# Uploading packages for 19th Nov Session:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
session.sql("USE ROLE TRAINING_ROLE").collect()     # or TRAINING_ROLE
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

In [ ]:
select status from la_permit_data.public.permit_records

In [ ]:
df = session.sql("select status from la_permit_data.public.permit_records").to_pandas()

In [ ]:
records        = session.table("PERMIT_RECORDS").to_pandas()
census_tracts  = session.table("CENSUS_TRACTS").to_pandas()
contractors    = session.table("MASTER_LICENSE").to_pandas()
personnel      = session.table("PERSONNEL_DATA").to_pandas()
worker_comp    = session.table("WORKER_COMP").to_pandas()

In [ ]:
records.columns
census_tracts.columns

In [ ]:
pd.to_datetime(records["STATUS_DATE"])

In [ ]:
records["ISSUE_DATE"]

In [ ]:
records["duration"] = pd.to_datetime(records["STATUS_DATE"]) - (pd.to_datetime(records["ISSUE_DATE"]))
records["duration"].head()

In [ ]:

permits_per_day = records.groupby("ISSUE_DATE").size()

In [ ]:
records["ISSUE_DATE"] = pd.to_datetime(records["ISSUE_DATE"])
records["year"] = records["ISSUE_DATE"].dt.year
permits_per_day = records.groupby("year").size()
permits_per_day.plot()

In [ ]:
clean_records = records.dropna(subset=["LATITUDE_LONGITUDE"]).copy()

# Split WKT POINT string into coordinates
coords = (
    clean_records["LATITUDE_LONGITUDE"]
    .str.replace("POINT (","")
    .str.replace(")","")
    .str.split(" ")
)

# Assign longitude and latitude
clean_records["longitude"] = coords.str[0].astype(float)
clean_records["latitude"]  = coords.str[1].astype(float)

In [ ]:
st.map(clean_records[["latitude", "longitude"]].head(100000))

In [ ]:
#clean_records["TRACT"]
#clean_records.columns
#contractors.columns
census_tracts.head(5)

In [ ]:
contractors.head(5)

In [ ]:
clean_records.head(5)

In [ ]:
cols = session.sql("""
    SHOW COLUMNS IN TABLE la_permit_data.public.MASTER_LICENSE
""").to_pandas()

cols

In [ ]:
df = pd.merge(contractors,clean_records,left_on='LICENSE_NO',right_on='LICENSE_NUM')
#dfpoint = df[df["CENSUS_TRACT"] == "2679.01"]
#dfpoint

In [ ]:
top_5 = (df.groupby('BUSINESS_NAME')['CENSUS_TRACT'].nunique()
      .reset_index(name='num_tracts')
      .sort_values('num_tracts', ascending=False)
      .head(5))

top_5_business_names = top_5['BUSINESS_NAME'].tolist()
df_top5 = df[df['BUSINESS_NAME'].isin(top_5_business_names)]
st.map(df_top5[['latitude', 'longitude']].dropna())

In [ ]:
Bottom_100 = (df.groupby('BUSINESS_NAME')['CENSUS_TRACT'].nunique()
      .reset_index(name='num_tracts')
      .sort_values('num_tracts', ascending=True)
      .head(100))

Bottom_100_business_names = Bottom_100['BUSINESS_NAME'].tolist()
df_Bottom100 = df[df['BUSINESS_NAME'].isin(Bottom_100_business_names)]
st.map(df_Bottom100[['latitude', 'longitude']].dropna())

In [ ]:
top_5
Bottom_100

In [ ]:
session.sql("SHOW TABLES").show()

In [ ]:
tract_counts = (
    df.groupby(['CENSUS_TRACT', 'CONTRACTOR_BUSINESS_NAME',"BUSINESS_TYPE"])
      .size()
      .reset_index(name='permit_count')
)
tract_counts

In [ ]:
tract_totals = (
    tract_counts.groupby('CENSUS_TRACT')['permit_count']
    .sum()
    .reset_index(name='total_permits')
)

# Merge totals back to contractor counts
tract_counts = tract_counts.merge(tract_totals, on='CENSUS_TRACT')

# Calculate share
tract_counts['share'] = tract_counts['permit_count'] / tract_counts['total_permits']
tract_counts

In [ ]:
top_shares_overall = tract_counts.sort_values('share', ascending=False)
top_shares_overall = top_shares_overall[top_shares_overall["total_permits"]>100]
top_shares_overall
#top_shares_overall[['CENSUS_TRACT', 'CONTRACTOR_BUSINESS_NAME', 'share','total_permits']]

In [ ]:
#top_shares_overall[["permit_count","total_permits"]].head(5).plot(kind='bar')
data = top_shares_overall[["BUSINESS_TYPE",
                           "permit_count", "total_permits"]].head(5)
data = data.set_index("BUSINESS_TYPE")
plt.figure(figsize=(10, 6))
data.plot(kind='bar')

plt.title("Top 5 Permit Shares")
plt.xlabel("Tracts")
plt.ylabel("Count")


In [ ]:
names = top_shares_overall["BUSINESS_TYPE"].unique()
print(names)

In [ ]:
df_counts = (
    df.groupby(['CENSUS_TRACT', 'BUSINESS_TYPE'])
      .size()  # counts rows (permits)
      .reset_index(name='permit_count')  # new column for counts
)
top_businesses = (
    df_counts.sort_values(['CENSUS_TRACT', 'permit_count'], ascending=[True, False])
      .groupby('CENSUS_TRACT')
      .head(1)  # top 5 business types per tract
      .reset_index(drop=True)
)
top_businesses[['CENSUS_TRACT', 'BUSINESS_TYPE', 'permit_count']]
bar_data = top_businesses.pivot(
    index='CENSUS_TRACT',
    columns='BUSINESS_TYPE',
    values='permit_count'
).fillna(0)

bar_data.head(1).plot(kind='bar', stacked=True, figsize=(12,6))
plt.title("Top Business Types per Tract by Share")
plt.xlabel("Census Tract")
plt.ylabel("Share")
plt.tight_layout()
plt.show()


In [ ]:
df_corp = df[df["BUSINESS_TYPE"] == "Corporation"]
df_part = df[df["BUSINESS_TYPE"] == "Partnership"]
df_lim = df[df["BUSINESS_TYPE"] == "Limited Liability"]
df_joint = df[df["BUSINESS_TYPE"] == "JointVenture"]
df_sole = df[df["BUSINESS_TYPE"] == "Sole Owner"]

In [ ]:
df_counts = (df.groupby(['BUSINESS_TYPE', 'CENSUS_TRACT']).size().reset_index(
    name='permit_count'))

In [ ]:
top3_per_type = (
    df_counts.sort_values(['BUSINESS_TYPE', 'permit_count'], ascending=[True, False])
             .groupby('BUSINESS_TYPE')
             .head(3)  # top 3 per business type
             .reset_index(drop=True))

In [ ]:
top3_per_type[['BUSINESS_TYPE', 'CENSUS_TRACT', 'permit_count']]

In [ ]:
bar_data = top3_per_type.pivot(index='CENSUS_TRACT', 
                               columns='BUSINESS_TYPE', 
                               values='permit_count').fillna(0)

bar_data.plot(kind='bar')
plt.title('Top 3 Business Types by Permit Count per Census Tract')
plt.xlabel('Census Tract')
plt.ylabel('Permit Count')
plt.legend(title='Business Type')
plt.show()

In [ ]:
df["VALUATION"] = (
    df["VALUATION"]
    .astype(str)                                  
    .str.replace("$", "", regex=False)             
    .str.replace(",", "", regex=False)             
    .str.strip()                                   
    .replace("None", None)                         
    .replace("", None)                             
)

df["VALUATION"] = pd.to_numeric(df["VALUATION"], errors="coerce")

In [ ]:
df["VALUATION"]

In [ ]:
df_summary = df.groupby('CENSUS_TRACT').agg(
    num_contractors=('CONTRACTOR_BUSINESS_NAME', 'nunique'),
    avg_cost=('VALUATION', 'mean')
).reset_index()
df_summary.sort_values("avg_cost",ascending=False)

In [ ]:
#df_summary.plot(kind='scatter',x="num_contractors",y="avg_cost")

plt.scatter(df_summary["num_contractors"],df_summary["avg_cost"])

plt.xlabel("Number of Contractors")
plt.ylabel("Average Cost (10s of Million $)")
plt.title("Correlation: Number of Contractors vs Average Cost for Each Tract")

plt.show()

In [ ]:
df.columns

In [ ]:
df_permit = df.groupby(['PERMIT_TYPE']).agg(
    num_contractors=('BUSINESS_NAME', 'nunique')).reset_index()
df_permit

In [ ]:

plt.bar(df_permit["PERMIT_TYPE"], df_permit["num_contractors"])
plt.xlabel("Permit Type")
plt.ylabel("Number of Contractors")
plt.title("Contractors by Permit Type")
plt.xticks(rotation=45, ha="right")

In [ ]:
df["year"]

In [ ]:
df_years = df.groupby("year").agg(
    permit_count=('PERMIT_TYPE', 'count')
).reset_index()
df_years

In [ ]:
#plt.figure(figsize=(10,6))
plt.plot(df_years['year'], df_years['permit_count'], marker='o')
plt.xlabel('Year')
plt.ylabel('Number of Permits')
plt.title('Permit Counts Per Year')
plt.grid(True)
plt.xticks(df_years['year'])  # ensures all years show
#plt.tight_layout()
#plt.show()

In [ ]:
# streamlit_dashboard_snowflake.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# -----------------------------
# 1. Load data from Snowflake Lab session
# -----------------------------
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Set role, database, schema
session.sql("USE ROLE TRAINING_ROLE").collect()
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

# Load tables
records = session.table("PERMIT_RECORDS").to_pandas()
census_tracts = session.table("CENSUS_TRACTS").to_pandas()
contractors = session.table("MASTER_LICENSE").to_pandas()
personnel = session.table("PERSONNEL_DATA").to_pandas()
worker_comp = session.table("WORKER_COMP").to_pandas()

# -----------------------------
# 2. Preprocessing
# -----------------------------
# Convert permit dates
records['ISSUE_DATE'] = pd.to_datetime(records['ISSUE_DATE'], errors='coerce')
records['YEAR'] = records['ISSUE_DATE'].dt.year

# Ensure numeric valuation
records['VALUATION'] = (
    records['VALUATION']
    .astype(str)
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)
records['VALUATION'] = pd.to_numeric(records['VALUATION'], errors='coerce')

# -----------------------------
# 3. Sidebar filters
# -----------------------------
st.sidebar.title("Filters")

# Year slider
year_range = st.sidebar.slider(
    "Select Year Range",
    min_value=2013,
    max_value=2023,
    value=(2013, 2023)
)

# Permit type filter
permit_options = records['PERMIT_TYPE'].dropna().unique().tolist()
selected_permits = st.sidebar.multiselect(
    "Select Permit Types",
    options=permit_options,
    default=permit_options
)

# Business type filter
business_options = contractors['BUSINESS_TYPE'].dropna().unique().tolist()
selected_businesses = st.sidebar.multiselect(
    "Select Business Types",
    options=business_options,
    default=business_options
)

# Merge records with contractor info
# Assuming 'CONTRACTOR_BUSINESS_NAME' is the key linking records to contractors
merged_df = records.merge(
    contractors[['BUSINESS_NAME', 'BUSINESS_TYPE']],
    right_on='BUSINESS_NAME',
    left_on='CONTRACTOR_BUSINESS_NAME'  # keep all permit records
)

# -----------------------------
# 4. Filter data based on selections
# -----------------------------
filtered_df = merged_df[
    (merged_df['YEAR'] >= year_range[0]) &
    (merged_df['YEAR'] <= year_range[1]) &
    (merged_df['PERMIT_TYPE'].isin(selected_permits)) &
    (merged_df['BUSINESS_TYPE'].isin(selected_businesses))
]

st.title("Los Angeles Building Permit Dashboard")

# -----------------------------
# 5. Permit Types vs Contractor Counts
# -----------------------------
st.subheader("Contractors by Permit Type")

df_permit = filtered_df.groupby('PERMIT_TYPE').agg(
    num_contractors=('CONTRACTOR_BUSINESS_NAME', 'nunique'),
    num_permits=('PERMIT_TYPE', 'count')
).reset_index().sort_values("num_contractors", ascending=False)

fig = px.bar(
    df_permit,
    x='PERMIT_TYPE',
    y='num_contractors',
    color='num_permits',
    color_continuous_scale='Viridis',
    labels={'num_contractors': 'Number of Contractors', 'PERMIT_TYPE': 'Permit Type'},
    title='Contractors per Permit Type (Color = Number of Permits)'
)
st.plotly_chart(fig, use_container_width=True)

# -----------------------------
# 6. Correlation: Number of Contractors vs Avg Cost
# -----------------------------
st.subheader("Contractors vs Average Valuation")

df_summary = filtered_df.groupby('CENSUS_TRACT').agg(
    num_contractors=('CONTRACTOR_BUSINESS_NAME', 'nunique'),
    avg_cost=('VALUATION', 'mean')
).reset_index()

fig2 = px.scatter(
    df_summary,
    x='num_contractors',
    y='avg_cost',
    hover_data=['CENSUS_TRACT'],
    trendline="ols",
    labels={'num_contractors': 'Number of Contractors', 'avg_cost': 'Average Valuation'},
    title='Number of Contractors vs Average Valuation by Census Tract'
)
st.plotly_chart(fig2, use_container_width=True)

# -----------------------------
# 7. Timeline of Permit Counts
# -----------------------------
st.subheader("Permit Counts Over Time")

df_time = filtered_df.groupby('YEAR').agg(
    permit_count=('PERMIT_TYPE', 'count')
).reset_index()

fig3 = px.line(
    df_time,
    x='YEAR',
    y='permit_count',
    markers=True,
    title='Permit Counts Per Year'
)
st.plotly_chart(fig3, use_container_width=True)

# -----------------------------
# 8. Map of Los Angeles Permits
# -----------------------------
st.subheader("Permit Locations in Los Angeles")

# Make sure your PERMIT_RECORDS table has LAT/LON columns
if 'LAT' in filtered_df.columns and 'LON' in filtered_df.columns:
    fig_map = px.scatter_mapbox(
        filtered_df,
        lat='LAT',
        lon='LON',
        color='BUSINESS_TYPE',
        size='VALUATION',
        hover_name='CONTRACTOR_BUSINESS_NAME',
        hover_data=['PERMIT_TYPE', 'YEAR'],
        zoom=9,
        height=600,
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    fig_map.update_layout(mapbox_style="carto-positron")
    st.plotly_chart(fig_map, use_container_width=True)
else:
    st.warning("Latitude and Longitude columns missing for mapping.")

# -----------------------------
# 9. Summary statistics
# -----------------------------
st.subheader("Summary Stats")
st.write(filtered_df.describe(include='all'))


In [ ]:
records.columns
contractors.columns

In [ ]:
records

In [ ]:
# la_permit_dashboard.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# -----------------------------
# 1. Load data from Snowflake Lab session
# -----------------------------
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Set role, database, schema
session.sql("USE ROLE TRAINING_ROLE").collect()
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

# Load tables
records = session.table("PERMIT_RECORDS").to_pandas()
census_tracts = session.table("CENSUS_TRACTS").to_pandas()
contractors = session.table("MASTER_LICENSE").to_pandas()

# -----------------------------
# 2. Preprocessing
# -----------------------------
# Convert permit dates
records['ISSUE_DATE'] = pd.to_datetime(records['ISSUE_DATE'], errors='coerce')
records['YEAR'] = records['ISSUE_DATE'].dt.year

# Ensure numeric valuation
records['VALUATION'] = (
    records['VALUATION']
    .astype(str)
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)
records['VALUATION'] = pd.to_numeric(records['VALUATION'], errors='coerce')


# -----------------------------
# 3. Sidebar filters
# -----------------------------
st.sidebar.title("Filters")

# Year slider
year_range = st.sidebar.slider(
    "Select Year Range",
    min_value=2013,
    max_value=2023,
    value=(2013, 2023),
    key="year_range_slider"
)

# Permit type filter
permit_options = records['PERMIT_TYPE'].dropna().unique().tolist()
selected_permits = st.sidebar.multiselect(
    "Select Permit Types",
    options=permit_options,
    default=permit_options
)

# Business type filter
business_options = contractors['BUSINESS_TYPE'].dropna().unique().tolist()
selected_businesses = st.sidebar.multiselect(
    "Select Business Types",
    options=business_options,
    default=business_options
)

# -----------------------------
# 4. Merge records with contractor info
# -----------------------------
merged_df = records.merge(
    contractors[['BUSINESS_NAME', 'BUSINESS_TYPE']],
    right_on='BUSINESS_NAME',
    left_on='CONTRACTOR_BUSINESS_NAME',
    how='left'
)

# Filter based on sidebar selections
filtered_df = merged_df[
    (merged_df['YEAR'] >= year_range[0]) &
    (merged_df['YEAR'] <= year_range[1]) &
    (merged_df['PERMIT_TYPE'].isin(selected_permits)) &
    (merged_df['BUSINESS_TYPE'].isin(selected_businesses))
]

# -----------------------------
# 5. Dashboard title
# -----------------------------
st.title("Los Angeles Building Permit Dashboard")

# -----------------------------
# 6. Contractors by Permit Type
# -----------------------------
st.subheader("Contractors by Permit Type")
df_permit = filtered_df.groupby('PERMIT_TYPE').agg(
    num_contractors=('CONTRACTOR_BUSINESS_NAME', 'nunique'),
    num_permits=('PERMIT_TYPE', 'count')
).reset_index().sort_values("num_contractors", ascending=False)

fig1 = px.bar(
    df_permit,
    x='PERMIT_TYPE',
    y='num_contractors',
    color='num_permits',
    color_continuous_scale='Viridis',
    labels={'num_contractors': 'Number of Contractors', 'PERMIT_TYPE': 'Permit Type'},
    title='Contractors per Permit Type (Color = Number of Permits)'
)
st.plotly_chart(fig1, use_container_width=True)

# -----------------------------
# 7. Contractors vs Average Valuation
# -----------------------------
st.subheader("Contractors vs Average Valuation")
df_summary = filtered_df.groupby('CENSUS_TRACT').agg(
    num_contractors=('CONTRACTOR_BUSINESS_NAME', 'nunique'),
    avg_cost=('VALUATION', 'mean')
).reset_index()

fig2 = px.scatter(
    df_summary,
    x='num_contractors',
    y='avg_cost',
    hover_data=['CENSUS_TRACT'],
    labels={'num_contractors': 'Number of Contractors', 'avg_cost': 'Average Valuation'},
    title='Number of Contractors vs Average Valuation by Census Tract'
)
st.plotly_chart(fig2, use_container_width=True)

# -----------------------------
# 8. Permit Counts Over Time
# -----------------------------
st.subheader("Permit Counts Over Time")
df_time = filtered_df.groupby('YEAR').agg(
    permit_count=('PERMIT_TYPE', 'count')
).reset_index()

fig3 = px.line(
    df_time,
    x='YEAR',
    y='permit_count',
    markers=True,
    title='Permit Counts Per Year'
)
st.plotly_chart(fig3, use_container_width=True)


# -----------------------------
# 10. Summary statistics
# -----------------------------
st.subheader("Summary Stats")
st.write(filtered_df.describe(include='all'))
